In [2]:
import gym
import random

In [3]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
# episodes = 10
# for episode in range (1,episodes+1):
#   state = env.reset()
#   done = False
#   score =0

#   while not done:
#     env.render()
#     action = random.choices([0,1])
#     n_state, reward, done, info = env.step(action[0])
#     score+=reward
#     if episode==10:
#       done=True
#   print('Episode:{} Score:{}'.format(episode,score))


In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


In [13]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [14]:
model = build_model(states, actions)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
    nb_actions = actions, nb_steps_warmup = 10, target_model_update=1e-2)
    return dqn

In [19]:
dqn = build_agent(model,actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 110s 11ms/step - reward: 1.0000
53 episodes - episode_reward: 186.642 [39.000, 200.000] - loss: 16.959 - mae: 46.967 - mean_q: 94.591

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 113s 11ms/step - reward: 1.0000
54 episodes - episode_reward: 186.667 [44.000, 200.000] - loss: 17.414 - mae: 43.220 - mean_q: 87.187

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 116s 12ms/step - reward: 1.0000
55 episodes - episode_reward: 181.618 [46.000, 200.000] - loss: 15.382 - mae: 42.053 - mean_q: 85.142

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 116s 12ms/step - reward: 1.0000
50 episodes - episode_reward: 199.040 [152.000, 200.000] - loss: 21.894 - mae: 47.374 - mean_q: 95.767

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 116s 12ms/step - reward:

In [20]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 175.000, steps: 175
Episode 6: reward: 125.000, steps: 125
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 119.000, steps: 119
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 142.000, steps: 142
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 183.000, steps: 183
Episode 24: reward: 191.000, steps: 191
Episode 25: reward: 

In [23]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 136.000, steps: 136
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 117.000, steps: 117
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 133.000, steps: 133
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


In [24]:
dqn.save_weights('dqn_weights.h5f',overwrite=True)

: 